### Summarize tidy tables
This script summarizes the water use and water suppy tidy tables generated by the *CreateUsageTable* and *CreateSupplyTable* scripts, respectively. Each table is then merged into a single dataframe to create a table listing water use and supply for each year/state combination. 

-----
#### Workflow
* Import and summarize use table on state, listing usage amounts by use class and source class (surface/groundwater)
* Import and summarize supply table, first on county so that amounts can be converted from mm to MGal/year.
* Summarize the county supply table to the state level, listing the total MGal/year of supply in each state

In [1]:
#Import libraries
import sys, os
import pandas as pd
import numpy as np

In [103]:
#Get file names; these files are created by the CreateUsageTable.py and CreateSupplyTable.py respectively
dataDir = '../../Data'
tidyuseFN = dataDir + os.sep + "UsageDataTidy.csv"
tidysupplyFN = dataDir + os.sep + "SupplyTableTidy.csv"
outCountyFN = dataDir + os.sep + "WaterByCounty.csv"
outStateFN = dataDir + os.sep + "WaterByState.csv"
outNationFN = dataDir + os.sep + "WaterBalanceData.csv"

#### Summarize USE table by county
Computes water usage for each county broken into each sector and source category.  

In [3]:
#Read in the usage table from the csv file
dfUse = pd.read_csv(tidyuseFN,dtype={'FIPS':np.str})

In [4]:
#Remove rows with irrigation and thermoelectric sub-classes
dropValues = ['Irrigation_Crop', 'Irrigation_Golf','ThermoElec_OnceThru', 'ThermoElec_Recirc']
dfUse = dfUse[~dfUse['UseClass'].isin(dropValues)]

In [102]:
#Convert amounts from MGal/day to MGal/year
dfUse['Amount'] = dfUse['Amount'] * 365

In [12]:
#Add STATEFIPS column to dfUse (as left most 2 characters of FIPS values)
dfUse['STATEFIPS'] = dfUse['FIPS'].str[:2]

In [53]:
#Pivot on YEAR and FIPS listing usage in sector/source categories
dfUseFIPS = dfUse.pivot_table(index=['YEAR','STATE','FIPS'],
                              values='Amount',
                              aggfunc='sum',
                              columns=['UseClass','SrcClass'])

In [61]:
#Flatten hierarchical column names
dfUseFIPS.columns = ['_'.join(col).strip() for col in dfUseFIPS.columns.values]

In [69]:
#Remove indices so values are available as columns
dfUseFIPS.reset_index(inplace=True)

In [71]:
dfUseFIPS.head(2)

,YEAR,STATE,FIPS,Aquaculture_Groundwater,Aquaculture_Surface,Domestic_Groundwater,Domestic_Surface,Industrial_Groundwater,Industrial_Surface,Irrigation_Groundwater,...,Livestock_Groundwater,Livestock_Surface,Mining_Groundwater,Mining_Surface,PublicSupply_Groundwater,PublicSupply_Surface,ThermoElec_Groundwater,ThermoElec_Surface,Total_Groundwater,Total_Surface
0,2000,AK,02013,NaN,NaN,0.01,0.0,0.0,2.25,0.0,...,NaN,NaN,0.0,0.0,0.02,0.98,0.0,0.0,0.03,3.23
1,2000,AK,02016,NaN,NaN,0.00,0.0,0.0,2.40,0.0,...,NaN,NaN,0.0,0.0,0.35,2.14,0.0,0.0,0.35,4.54


#### Import and summarize supply table by county

In [84]:
#Read in the supply table from the csv file
dfSupply = pd.read_csv(tidysupplyFN,dtype={'FIPS':np.str,'STATEFIPS':np.str})                    

In [88]:
#Compute supply as precipitation - evapotranspiration 
#(See https://www.fs.fed.us/rm/value/docs/spatial_distribution_water_supply.pdf)
# * Could also use total_runoff
# * Values are in mm/year and need to be adjusted to MGal/year by mulitplying by weighted area
dfSupply['Supply'] = dfSupply['pr'] - dfSupply['et']

In [89]:
#Summarize supply on YEAR and FIPS
'''We take the mean mm/year across points in a county and then 
   mulitply by county area to get volume (mm * m3). These values
   then need to by converted to MGal to give MGal/year
'''

#Compute mean runoff and supply on year and county
dfSupplyFIPS = dfSupply.groupby(('YEAR','STATEFIPS','FIPS','Area'))['total_runoff','Supply'].mean()

#Reset the index so Year, StateFIPS, FIPS, and AREA become columns again
dfSupplyFIPS.reset_index(inplace=True)

#Convert mm/Year * county area (m2) into MGal/year - to match use values
''' m = [mm] / 1000; 
    m * [m2] = m3;
   [m3] / 3785.41178 = 1 MGal'''
for param in ('total_runoff','Supply'):
    dfSupplyFIPS[param] = (dfSupplyFIPS[param] / 1000.0) * dfSupplyFIPS.Area / 3785.41178

In [90]:
dfSupplyFIPS.head(2)

,YEAR,STATEFIPS,FIPS,Area,total_runoff,Supply
0,2000,01,01001,1.565359e+09,164183.119550,136727.091260
1,2000,01,01003,5.250715e+09,747212.294697,697763.569535


### Join Use and Supply Tables on Year and FIPS

In [91]:
dfSupplyFIPS.columns.values

array(['YEAR', 'STATEFIPS', 'FIPS', 'Area', 'total_runoff', 'Supply'], dtype=object)

In [92]:
#Merge the two tables on YEAR and FIPS columns
dfAll = pd.merge(dfUseFIPS,dfSupplyFIPS, how='outer',on=['YEAR','FIPS'],left_index=True,right_index=True)

In [93]:
dfAll.columns.values

array(['YEAR', 'STATE', 'FIPS', 'Aquaculture_Groundwater',
       'Aquaculture_Surface', 'Domestic_Groundwater', 'Domestic_Surface',
       'Industrial_Groundwater', 'Industrial_Surface',
       'Irrigation_Groundwater', 'Irrigation_Surface',
       'Livestock_Groundwater', 'Livestock_Surface', 'Mining_Groundwater',
       'Mining_Surface', 'PublicSupply_Groundwater',
       'PublicSupply_Surface', 'ThermoElec_Groundwater',
       'ThermoElec_Surface', 'Total_Groundwater', 'Total_Surface',
       'STATEFIPS', 'Area', 'total_runoff', 'Supply'], dtype=object)

In [100]:
#Export to csv
dfAll.to_csv(outCountyFN, index=False, encoding='utf8')

In [99]:
#Summarize use data by year, state, and source type (surface vs groundwater)
dfStateUse = dfUse.pivot_table(index=['YEAR','SrcClass'],aggfunc='sum',columns=['UseClass'])
dfSt

NameError: name 'dfSt' is not defined